# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-21 19:08:08,130 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
from pyspark.sql.window import Window 
import pyspark.sql.functions as F


In [3]:
! hdfs dfs -put clickstream.csv


put: `clickstream.csv': File exists


In [4]:
df = se.read.option("delimiter", "\t").csv('clickstream.csv', header=True, inferSchema=True)


In [5]:
df.createOrReplaceTempView("my_table")

# Use Spark SQL to query the data
query = "SELECT * FROM my_table"
result = se.sql(query)
result.show()

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
|    562|       507|       event|    rabota|1695584181|
|    562|       507|       event|    rabota|1695584189|
|    562|       507|        page|      main|1695584194|
|    562|       507|       event|      main|1695584204|
|    562|       507|       event|      main|1695

In [19]:
query = """
with errors as (
    SELECT
        user_id, 
        session_id, 
        min(timestamp) as timestamp
    FROM my_table
    WHERE 1=1
        AND event_type like '%error%'
    GROUP BY 1,2
    ORDER BY timestamp
),
filtered_logs as (
    SELECT 
        t.* 
    FROM my_table t
    LEFT JOIN errors e ON e.user_id=t.user_id AND e.session_id=t.session_id
    WHERE 1=1
        AND ((e.user_id is not null AND e.timestamp>t.timestamp) OR (e.user_id is null))
    ORDER BY t.timestamp),
grouped_route as (
    SELECT 
        user_id,
        session_id,
        concat_ws('-',collect_list(event_page)) as user_route 
    FROM (
        SELECT 
            *
            , LAG(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) as prev_page
        FROM filtered_logs
    ) a
    WHERE (prev_page != event_page) OR prev_page is null
    GROUP BY 1,2)
SELECT 
    user_route as route,
    COUNT(*) as count 
FROM grouped_route
GROUP BY 1
ORDER BY count desc

"""
result = se.sql(query)
result.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|    main-news-rabota|  135|
|   main-bonus-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+
only showing t

In [7]:
df = se.read.option("delimiter", "\t").csv('clickstream.csv', header=True, inferSchema=True)

In [8]:

# df = df.where(col('user_id')==562)
first_error = df.where(df.event_type.like('%error%')).groupBy('user_id', 'session_id').agg(F.min('timestamp').alias('error_timestamp'))
first_error = first_error.selectExpr("user_id as user_id_error", "session_id as session_id_error", "error_timestamp")

In [9]:
filtered_logs = df.join(first_error, [df.user_id==first_error.user_id_error, df.session_id == first_error.session_id_error], how='left').\
where((F.col('error_timestamp')>F.col('timestamp')) | (F.col('error_timestamp').isNull()))\
.orderBy(F.col("timestamp"))

In [10]:
windowSpec  = Window.partitionBy("user_id", "session_id").orderBy("timestamp")
logs_with_lag = filtered_logs.withColumn("lag",F.lag("event_page",1).over(windowSpec)).where((F.col('event_page')!=F.col('lag'))|(F.col('lag').isNull())).orderBy('timestamp')


In [11]:
w = Window.partitionBy('user_id', 'session_id').orderBy('timestamp')
# using window with sorting in order to have the same output everytime
end_table = logs_with_lag.withColumn(
            'sorted_list', F.array_join(F.collect_list('event_page').over(w), "-")
        )\
        .groupBy('user_id', 'session_id')\
        .agg(F.max('sorted_list').alias('sorted_list')).groupBy('sorted_list').count().orderBy(F.desc('count'))
end_table.show(30)

+--------------------+-----+
|         sorted_list|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|    main-news-rabota|  135|
|   main-bonus-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+
only showing t

In [12]:
rdd = sc.textFile("clickstream.csv")
rdd.take(5)

['user_id\tsession_id\tevent_type\tevent_page\ttimestamp',
 '562\t507\tpage\tmain\t1695584127',
 '562\t507\tevent\tmain\t1695584134',
 '562\t507\tevent\tmain\t1695584144',
 '562\t507\tevent\tmain\t1695584147']

In [13]:
import shlex
rdd.map(lambda line: shlex.split(line)).take(10)

[['user_id', 'session_id', 'event_type', 'event_page', 'timestamp'],
 ['562', '507', 'page', 'main', '1695584127'],
 ['562', '507', 'event', 'main', '1695584134'],
 ['562', '507', 'event', 'main', '1695584144'],
 ['562', '507', 'event', 'main', '1695584147'],
 ['562', '507', 'wNaxLlerrorU', 'main', '1695584154'],
 ['562', '507', 'event', 'main', '1695584154'],
 ['562', '507', 'event', 'main', '1695584154'],
 ['562', '507', 'event', 'main', '1695584160'],
 ['562', '507', 'page', 'rabota', '1695584166']]